# Linear Regression

## Cleaning and Splitting (copied from initial_exploration)

In [15]:
import pandas as pd
df = pd.read_csv("suiciderates.csv")

#Making generations boolean
df_split_gen = df.copy()

def gen_x_bool(gen):
    if gen == "Generation X":
        return 1
    else:
        return 0

df_split_gen["Generation X"] = df_split_gen["generation"].map(gen_x_bool)

def silent_bool(gen):
    if gen == "Silent":
        return 1
    else:
        return 0

df_split_gen["Silent"] = df_split_gen["generation"].map(silent_bool)

def mil_bool(gen):
    if gen == "Millenials":
        return 1
    else:
        return 0

df_split_gen["Millenials"] = df_split_gen["generation"].map(mil_bool)

def boom_bool(gen):
    if gen == "Boomers":
        return 1
    else:
        return 0

df_split_gen["Boomers"] = df_split_gen["generation"].map(boom_bool)

def g_i_bool(gen):
    if gen == "G.I. Generation":
        return 1
    else:
        return 0

df_split_gen["G.I. Generation"] = df_split_gen["generation"].map(g_i_bool)

def z_bool(gen):
    if gen == "Generation Z":
        return 1
    else:
        return 0

df_split_gen["Generation Z"] = df_split_gen["generation"].map(z_bool)

# Drop the original generation column
drop_gen = df_split_gen.copy()
drop_gen.drop(labels="generation", axis='columns', inplace=True)

# Changing sex to a boolean
df_sex = drop_gen.copy()
df_sex["sex"].replace("female", 0,inplace=True)
df_sex["sex"].replace("male", 1, inplace=True)

# Rename it to male for easier understanding
df_sex.rename(columns = {'sex':'male'}, inplace = True)

# Changing age to a boolean
age_bool = df_sex.copy()

def g1_bool(age):
    if age == "5-14 years":
        return 1
    else:
        return 0

age_bool["5-14 years"] = age_bool["age"].map(g1_bool)

def g2_bool(age):
    if age == "15-24 years":
        return 1
    else:
        return 0

age_bool["15-24 years"] = age_bool["age"].map(g2_bool)

def g3_bool(age):
    if age == "25-34 years":
        return 1
    else:
        return 0

age_bool["25-34 years"] = age_bool["age"].map(g3_bool)

def g4_bool(age):
    if age == "35-54 years":
        return 1
    else:
        return 0

age_bool["35-54 years"] = age_bool["age"].map(g4_bool)

def g5_bool(age):
    if age == "55-74 years":
        return 1
    else:
        return 0

age_bool["55-74 years"] = age_bool["age"].map(g5_bool)

def g6_bool(age):
    if age == "75+ years":
        return 1
    else:
        return 0

age_bool["75+ years"] = age_bool["age"].map(g6_bool)

# Drop HDI for year
drop_hdi = age_bool.copy()
drop_hdi.drop(labels="HDI for year", axis='columns', inplace=True)

# Convert GDP to numerical values
gdp_to_num = drop_hdi.copy()
# rename the columns due to formatting issues
new_cols = gdp_to_num.columns.values
new_cols[8] = 'gdp_for_year $'
new_cols[9] = 'gdp_per_capita $'
gdp_to_num.columns = new_cols

#Change the datatype of gdp_for_year
gdp_to_num['gdp_for_year $'] = gdp_to_num.iloc[:,8].str.replace(',', '').astype(float)

from sklearn.model_selection import train_test_split

data2 = gdp_to_num.copy()
train_set, test_set = train_test_split(data2, test_size=0.2, random_state=123)
print("traning set length:", len(train_set),"\ntest set length:", len(test_set))
print(data2.info())

traning set length: 22256 
test set length: 5564
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 22 columns):
country              27820 non-null object
year                 27820 non-null int64
male                 27820 non-null int64
age                  27820 non-null object
suicides_no          27820 non-null int64
population           27820 non-null int64
suicides/100k pop    27820 non-null float64
country-year         27820 non-null object
gdp_for_year $       27820 non-null float64
gdp_per_capita $     27820 non-null int64
Generation X         27820 non-null int64
Silent               27820 non-null int64
Millenials           27820 non-null int64
Boomers              27820 non-null int64
G.I. Generation      27820 non-null int64
Generation Z         27820 non-null int64
5-14 years           27820 non-null int64
15-24 years          27820 non-null int64
25-34 years          27820 non-null int64
35-54 years          27820 non-null i

## Perform a linear regression:

In [17]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
working_set = train_set.copy()

### Initial Features:
- I am going to choose suicides_no as my initial Y value since that is what I want to predict.
- I am going to choose male (or sex) as my initial feature for X. This is because I would expect gender to be a big factor in explaining number of suicides.

### Regression with one feature, "male"

In [26]:
X = working_set[["male"]]
Y = working_set["suicides_no"]
reg.fit(X, Y)

print("The bias is " , reg.intercept_)
print("The feature coefficients are ", reg.coef_)
print("R-squared equals: ", reg.score(X,Y))

The bias is  112.18620504081082
The feature coefficients are  [263.35165397]
R-squared equals:  0.020790546884134398


### Comments:
The bias is kind of high, but the feature coefficient is very large, meaning that gender plays a large role in number of suicides. The R-squared value; however, is very low which is not good. We want to be able to explain more of our variation in Y with our X values.

### Regression with two features, "male" and "gdp_for_year"

In [19]:
X = working_set[["male", "gdp_for_year $"]]
Y = working_set["suicides_no"]
reg.fit(X, Y)

print("The bias is " , reg.intercept_)
print("The feature coefficients are ", reg.coef_)
print("R-squared equals: ", reg.score(X,Y))

The bias is  -8.757882384200286
The feature coefficients are  [2.60346548e+02 2.75162435e-10]
R-squared equals:  0.20746355880458744


### Comments:
The bias is -8 which concerns me because no one should be in the negatives for likliness of suicide. Also, the feature coefficient of gdp_for_year is very small, meaning that the feature is probably insignificant. On the other hand, the R-squared value is roughly 20% higher than the initial value. I think we should consider keeping gdp_for_year in the model, but I'm not set on it.

### Regression using all features

In [25]:
#Run all without country and country-year
X = working_set[["year", "male", "5-14 years", "15-24 years", 
                 "25-34 years", "35-54 years", "55-74 years", 
                 "75+ years", "population", "suicides/100k pop", 
                 "gdp_for_year $", "gdp_per_capita $", "Generation X", 
                 "Silent", "Millenials", "Boomers", "G.I. Generation", 
                 "Generation Z"]]
Y = working_set["suicides_no"]
reg.fit(X, Y)

print("The bias is " , reg.intercept_)
print("The feature coefficients are ", reg.coef_)
print("R-squared equals: ", reg.score(X,Y))

The bias is  1121.742527479717
The feature coefficients are  [-6.80631060e-01  7.81270921e+01 -6.67873034e+01 -7.49764304e+00
  1.99822015e+01  8.32979531e+01  4.57535828e+01 -7.47487910e+01
  1.43260053e-04  1.34465227e+01 -6.49357642e-12  8.26123785e-04
 -2.69063629e+01  1.34917457e+01 -7.70974155e+00  1.38490477e+01
 -1.41936912e+01  2.14690022e+01]
R-squared equals:  0.47544088711326377


### Comments:
I first tested all of the columns, that were numerical or boolean, in my model. I could not use countries because there were way too many to convert them to boolean values. There is definitely overfitting here, but it's a better place to start. I will remove some features next. The R-squared got significantly higher, but so did the bias. Now, we can remove features and be able to better tell what really is explaining the variation in y.

### Regression with 2 features, "population" and "gdp_per_capita"

In [21]:
#Take out insignificant features
X = working_set[["population", "suicides/100k pop", "gdp_per_capita $"]]
Y = working_set["suicides_no"]
reg.fit(X, Y)

print("The bias is " , reg.intercept_)
print("The feature coefficients are ", reg.coef_)
print("R-squared equals: ", reg.score(X,Y))

The bias is  -217.34950686597102
The feature coefficients are  [1.43784527e-04 1.44500347e+01 6.28569766e-04]
R-squared equals:  0.469394887609627


In [32]:
#Take out insignificant features
X = working_set[["gdp_per_capita $"]]
Y = working_set["suicides_no"]
reg.fit(X, Y)

print("The bias is " , reg.intercept_)
print("The feature coefficients are ", reg.coef_)
print("R-squared equals: ", reg.score(X,Y))

# Plot outputs
import matplotlib.pyplot as plt
plt.scatter(X, Y,  color='black')
#plt.plot(X, Y, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())

plt.show()

The bias is  194.41060463992696
The feature coefficients are  [0.00290469]
R-squared equals:  0.003640233276967941


### Comments:
After removing all the insignificant features, we were left with a much better R-squared value than we started with and way less predictors than the last model. The R-squared value is still less than 50% which is not ideal, but without being able to factor in country, this is the best model. The bias is also not ideal.

Note: "suicides/100k pop" was a significant feature, but it would create ...

### Testing the Data:

In [22]:
#X = test_set[["population", "suicides/100k pop", "gdp_per_capita $"]]
X = test_set[["population", "gdp_per_capita $"]]
Y = test_set["suicides_no"]
reg.fit(X, Y)

print("The bias is " , reg.intercept_)
print("The feature coefficients are ", reg.coef_)
print("R-squared equals: ", reg.score(X,Y))

The bias is  -4.170600025119256
The feature coefficients are  [1.32119416e-04 2.15423623e-05]
R-squared equals:  0.383283384911531


### Comments:
All of the metrics on the test set are virtually the same as the training set. The bias actually went down and the R-squared got a little better which is a good sign. This is a fairly good model.
### Overall Comments:
Gender was not a large predictor like I thought it would be; however, population and gdp_per_capita were significant predictors.